In [48]:
import pandas as pd

data = pd.read_excel(r'/Users/allan/Desktop/Tripadvisor Restaurant Data Jan 2017.xlsx', error_bad_lines=False);
data_text = data[['Review']]

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(data_text['Review'].values.astype('U'))

In [57]:
doc_term_matrix

'''Each of 20k documents is represented as 14546 dimensional 
vector, which means that our vocabulary has 14546 words.'''

<24316x16100 sparse matrix of type '<class 'numpy.int64'>'
	with 809928 stored elements in Compressed Sparse Row format>

In [58]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [60]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 20 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print('\n')

Top 20 words for topic #0:
['atmosphere', 'experience', 'good', 'pizza', 'friendly', 'italian', 'dinner', 'recommend', 'staff', 'amazing', 'menu', 'new', 'delicious', 'excellent', 'best', 'place', 'restaurant', 'service', 'great', 'food']


Top 20 words for topic #1:
['fries', 'best', 'night', 'beer', 'little', 'pizza', 'wait', 'drinks', 'nice', 'time', 'burger', 'just', 'really', 'like', 'service', 'bar', 'great', 'food', 'place', 'good']


Top 20 words for topic #2:
['place', 'tasty', 'soup', 'friendly', 'enjoyed', 'fresh', 'wine', 'dinner', 'menu', 'salad', 'chicken', 'lunch', 'excellent', 'nice', 'restaurant', 'delicious', 'great', 'service', 'food', 'good']


Top 20 words for topic #3:
['like', 'friendly', 'best', 'soon', 'review', 'hope', 'really', 'enjoyed', 'staff', 'visit', 'place', 'restaurant', 'experience', 'thank', 'time', 'breakfast', 'great', 'service', 'good', 'food']


Top 20 words for topic #4:
['took', 'didn', 'minutes', 'service', 'like', 'dinner', 'waiter', 'good',

# NMF

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(data_text['Review'].values.astype('U'))

In [62]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=5, random_state=42)
nmf.fit(doc_term_matrix )

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [63]:
for i,topic in enumerate(nmf.components_):
    print(f'Top 20 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print('\n')

Top 20 words for topic #0:
['enjoyed', 'got', 'steak', 'order', 'came', 'chicken', 'wait', 'best', 'did', 'delicious', 'menu', 'dinner', 'experience', 'like', 'meal', 'just', 'ordered', 'table', 'time', 'restaurant']


Top 20 words for topic #1:
['visit', 'fun', 'loved', 'awesome', 'love', 'fantastic', 'highly', 'delicious', 'bar', 'recommend', 'drinks', 'amazing', 'excellent', 'friendly', 'staff', 'atmosphere', 'place', 'service', 'food', 'great']


Top 20 words for topic #2:
['reasonable', 'tasty', 'restaurant', 'little', 'atmosphere', 'bit', 'quality', 'selection', 'pretty', 'friendly', 'price', 'excellent', 'prices', 'lunch', 'place', 'really', 'nice', 'service', 'food', 'good']


Top 20 words for topic #3:
['slices', 'like', 've', 'italy', 'salad', 'just', 'friendly', 'sauce', 'delicious', 'toppings', 'fresh', 'cheese', 'pizzas', 'pasta', 'slice', 'best', 'italian', 'crust', 'place', 'pizza']


Top 20 words for topic #4:
['benedict', 'wait', 'fresh', 'french', 'ate', 'quick', 'bac

# Word Embedding
data preprocessing(no lematize)

In [2]:
import pandas as pd

data = pd.read_excel(r'/Users/allan/Desktop/Tripadvisor Restaurant Data Jan 2017.xlsx', error_bad_lines=False);
data_text = data[['Review']]
data_text['index'] = data_text.index
documents = data_text

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/allan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) 
#lemmatize: change past to present,stemmer: change to root form

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

'''Tokenization: Split the text into sentences and the sentences into words. 
Lowercase the words and remove punctuation.
Words that have fewer than 3 characters are removed.
All stopwords are removed.'''

'Tokenization: Split the text into sentences and the sentences into words. \nLowercase the words and remove punctuation.\nWords that have fewer than 3 characters are removed.\nAll stopwords are removed.'

In [6]:

processed_docs = documents['Review'].map(preprocess)

In [7]:
processed_docs[:10]

0    [second, visit, morton, steakhouse, vegas, loc...
1    [restaurant, alley, reminded, little, restaura...
2    [went, sunday, brunch, great, time, arrive, ti...
3    [freemans, brunch, pretty, exciting, menu, sma...
4    [dinner, meeting, colleagues, ambience, perfec...
5    [great, expectations, went, christmas, nice, d...
6    [gorgeous, place, loved, coming, trip, york, p...
7    [hidden, soho, clearly, popular, turs, night, ...
8    [food, excellent, course, expensive, restauran...
9    [visit, steakhouse, york, menu, contained, gre...
Name: Review, dtype: object

# Word Embedding
Word embedding model

In [8]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

path = get_tmpfile("word2vec.model")
model = Word2Vec(processed_docs, size=150, window=10, min_count=2, workers=10, iter=10)
words = list(model.wv.vocab)

# Word Embedding 
Generate dictionary only use one word

In [119]:
s_list=[]
for i, word in enumerate(words):
    score = model.similarity(word,'service')
    if score > 0.38:
        s_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [120]:
a_list=[]
for i, word in enumerate(words):
    score = model.similarity(word,'atmosphere')
    if score > 0.4:
        a_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [121]:
v_list=[]
for i, word in enumerate(words):
    score = model.similarity(word,'value')
    if score > 0.4:
        v_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [122]:
f_list=[]
for i, word in enumerate(words):
    score = model.similarity(word,'shrimp')
    if score > 0.45:
        f_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


# Word Embedding
Implicit Recommendation

In [156]:
for i, word in enumerate(words):
    score = model.similarity(word,'recommend')
    if score > 0.45:
        print(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


recommend
recommended
suggest
rated
screams
coordinated
recomend
recommends
repertoire
recommendable
recomended
reccomend
mildly
nutshell
touted
interruptions
acclaimed
commendable
absurd
pounding
wages
columbian
genres
bradbspurs
mars


In [163]:
for i, word in enumerate(words):
    score = model.similarity(word,'commendable')
    if score > 0.45:
        print(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


competent
recommend
knowledge
professional
recommended
rate
pretentious
approachable
courteous
buvette
exceptional
compromise
preparation
deliver
experienced
hardworking
extremely
encourage
rating
rated
screams
unfriendly
exceptionally
unbelievable
polite
highest
hovering
encountered
faultless
accurate
suggesting
memorable
elegance
butts
stars
genuine
repeat
coordinated
recomend
mark
fault
courtesy
terms
sensational
punctual
necessarily
patch
exemplary
grind
reflective
impressions
accomodating
extraordinary
gratifying
recognizes
minimal
lacking
exorbitant
friendliness
bothersome
eager
trained
outgoing
responsive
recommending
hospitable
sufficiently
incredibly
bellies
aways
befitting
caring
megan
disinterested
personality
standpoint
richard
chasing
brusque
overbearing
ratio
unprofessional
respectful
exudes
surprises
humor
excellence
unobtrusive
cleanliness
flawless
deserves
enthusiastic
receives
creates
panda
measure
attentiveness
rarely
recommendable
engage
snooty
execution
friendlier


In [124]:
df_word = pd.concat([pd.DataFrame({'food': f_list}), pd.DataFrame({'service':s_list}), pd.DataFrame({'value':v_list}), pd.DataFrame({'atmosphere':a_list})], axis=1)
df_word.to_excel(r'/Users/allan/Desktop/dictionary_v1.xlsx')

# skills to counter value in list

In [52]:
a = [1, 2, 3, 1, 1, 2]
b = [1, 2, 3, 4, 5, 9, 10]

In [54]:
test_list=[]
for x in a:
    test_list.append(x)

In [57]:
for x in b:
    test_list.append(x)

In [63]:
from collections import Counter
Counter(test_list)

Counter({1: 4, 2: 3, 3: 2, 4: 1, 5: 1, 9: 1, 10: 1})

In [70]:
dict = {}
for key in test_list:
    dict[key] = dict.get(key, 0) + 1
print(dict)

{1: 4, 2: 3, 3: 2, 4: 1, 5: 1, 9: 1, 10: 1}


# Word Embedding
Service

In [2]:
service_list = []
service_word_list = []
for i, word in enumerate(words):
    score = model.similarity(word,'service')
    if score > 0.38:
        service_list.append(word)

NameError: name 'words' is not defined

In [3]:
for i, word in enumerate(words):
    for x in service_list:
            service_score = model.similarity(word,x)
            if service_score > 0.38:
                service_word_list.append(word)

NameError: name 'words' is not defined

In [383]:
"""rank and see the result"""
service_result = pd.value_counts(service_word_list)
s = service_result.head(300)
s = pd.DataFrame(s)
s = s._stat_axis.values.tolist()

service       24
slammed       21
personable    21
incredibly    21
chummy        21
              ..
proactive     12
obliging      12
exudes        12
present       12
aided         12
Length: 300, dtype: int64

# Word Embedding
Food(append food)

In [142]:
food_list = []
food_word_list = []
for i, word in enumerate(words):
    score = model.similarity(word,'shrimp')
    if score > 0.45:
        food_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.


In [143]:
for i, word in enumerate(words):
    for x in food_list:
            food_score = model.similarity(word,x)
            if food_score > 0.45:
                food_word_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [406]:
"""rank and see the result"""
food_result = pd.value_counts(food_word_list)
f = food_result.head(3000)
f = pd.DataFrame(f)
f = f._stat_axis.values.tolist()

shrimp          814
shrimps         806
blackened       801
prawn           800
tiger           793
               ... 
scolded         328
commonly        328
sierra          328
marshmallows    328
plaintain       328
Length: 3000, dtype: int64

# Word Embedding
Value

In [145]:
value_list = []
value_word_list = []
for i, word in enumerate(words):
    score = model.similarity(word,'value')
    if score > 0.40:
        value_list.append(word)


/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.


In [146]:
for i, word in enumerate(words):
    for x in value_list:
            value_score = model.similarity(word,x)
            if value_score > 0.40:
                value_word_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [380]:
"""rank and see the result"""
value_result = pd.value_counts(value_word_list)
v = value_result.head(200)
v = pd.DataFrame(v)
v = v._stat_axis.values.tolist()

value            109
eighteen         100
premium           99
quantity          98
comparison        91
                ... 
discounted        60
cheapest          60
offbeat           60
extraordinary     60
metric            60
Length: 200, dtype: int64

# Word Embedding
Atmosphere

In [322]:
atmosphere_list = []
atmosphere_word_list = []
for i, word in enumerate(words):
    score = model.similarity(word,'atmosphere')
    if score > 0.45:
        atmosphere_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.


In [323]:
for i, word in enumerate(words):
    for x in atmosphere_list:
            atmosphere_score = model.similarity(word,x)
            if atmosphere_score > 0.40:
                atmosphere_word_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [324]:
len(atmosphere_word_list)

152727

In [386]:
"""rank and see the result"""
atmosphere_result = pd.value_counts(atmosphere_word_list)
a = atmosphere_result.head(300)
a = pd.DataFrame(a)
a = a._stat_axis.values.tolist()

environment     70
atmosphere      70
vibe            70
stuffy          69
surroundings    69
                ..
singers         54
candlelight     54
themed          54
hubbub          54
dancer          54
Length: 300, dtype: int64

# Repurchase

In [20]:
repurchase_list = []
repurchase_word_list = []
for i, word in enumerate(words):
    score = model.similarity(word,'return')
    if score > 0.45:
        print(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.


visit
recommend
come
enjoyable
return
captured
returning
stopping
keeper
revisit
woukd
beth
commendable
carol


In [19]:
a = pd.DataFrame(repurchase_list, columns = ['most_similar_Word'])
a.to_csv('/Users/allan/Desktop/repurchase/1_result.csv')

In [201]:
for i, word in enumerate(words):
    for x in repurchase_list:
            repurchase_score = model.similarity(word,x)
            if repurchase_score > 0.40:
                repurchase_word_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [203]:
"""rank and see the result"""
repurchase_result = pd.value_counts(repurchase_word_list)
repurchase_result.head(50)

returning        12
return           12
repeat           11
myers            10
adriano           9
stopping          9
carol             9
users             8
deserving         8
recommending      8
jennifer          8
regret            8
mamas             8
appreciative      8
priority          8
ironing           8
choosing          8
weeklong          8
fran              7
circumstance      7
arise             7
markteam          7
revisit           7
trips             7
cowfish           7
dine              7
conscious         7
honored           7
trendiest         7
carla             7
gouge             7
sean              7
coming            7
rethink           7
travis            7
rico              7
noho              7
giorgio           7
nora              7
overjoyed         7
complimenting     7
satisfactory      7
pass              7
joining           7
geremew           7
future            7
waistline         7
lolinda           7
susann            7
shall             7


In [330]:
repurchase_list

['visit',
 'coming',
 'come',
 'return',
 'plan',
 'repeat',
 'returning',
 'stopping',
 'keeper',
 'revisit',
 'beth',
 'bradbspurs']

# Implicit

In [241]:
recommend_list = []
recommend_word_list = []
for i, word in enumerate(words):
    score = model.similarity(word,'recommend')
    if score > 0.45:
        recommend_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.


In [242]:
#for i, word in enumerate(words):
    #score = model.similarity(word,'tasty')
    #if score > 0.50:
        #recommend_list.append(word)

In [243]:
for i, word in enumerate(words):
    for x in recommend_list:
            recommend_score = model.similarity(word,x)
            if recommend_score > 0.40:
                recommend_word_list.append(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [391]:
"""rank and see the result"""
recommend_result = pd.value_counts(recommend_word_list)
r = recommend_result.head(300)
r = pd.DataFrame(r)
r = r._stat_axis.values.tolist()

recomended       25
recommend        25
recommendable    24
touted           24
screams          23
                 ..
wayfarer         11
worlds           11
fudge            11
concentrating    11
privilege        11
Length: 300, dtype: int64

In [356]:
import pickle
with open('negation.pickle', 'wb') as b:
    pickle.dump(f,b)
    pickle.dump(s,b)
    pickle.dump(v,b)
    pickle.dump(a,b)

In [327]:
df_dictionary = pd.DataFrame()
df_dictionary['Food'] = f
df_dictionary['Service'] = s
df_dictionary['Value'] = v
df_dictionary['Atmosphere'] = a
df_dictionary['Implicit Recommendation'] = r

In [329]:
df_dictionary.to_excel(r'/Users/allan/Desktop/dictionary_v2.xlsx',index=False)

In [377]:
new_f_list.append('food')

## dictionary v3

In [409]:
new_f_list = f._stat_axis.values.tolist()
new_s_list = s._stat_axis.values.tolist()
new_v_list = v._stat_axis.values.tolist()
new_a_list = a._stat_axis.values.tolist()
new_r_list = r._stat_axis.values.tolist()

In [444]:
df_dic_v3 = pd.concat([pd.DataFrame({'food': new_f_list}), pd.DataFrame({'service':new_s_list}), pd.DataFrame({'value':new_v_list}), pd.DataFrame({'atmosphere':new_a_list}), pd.DataFrame({'Implicit Recommendation':new_r_list})], axis=1)
df_dic_v3.to_excel(r'/Users/allan/Desktop/work/dictionary_v3.xlsx',index=False)
df_dic_v3

,food,service,value,atmosphere,Implicit Recommendation
0,shrimp,service,value,environment,recomended
1,shrimps,slammed,eighteen,atmosphere,recommend
2,blackened,personable,premium,vibe,recommendable
3,prawn,incredibly,quantity,stuffy,touted
4,tiger,chummy,comparison,surroundings,screams
...,...,...,...,...,...
2995,scolded,NaN,NaN,NaN,NaN
2996,commonly,NaN,NaN,NaN,NaN
2997,sierra,NaN,NaN,NaN,NaN
2998,marshmallows,NaN,NaN,NaN,NaN


# Sentiment Dictionary

In [331]:
posdict = []
for i, word in enumerate(words):
    score = model.similarity(word,'great')
    if score > 0.45:
        print(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


great
perfect
terrific
good
super
loved
delicious
excellent
enjoyed
nice
lovely
lots
cool
love
wonderful
amazing
fantastic
relaxed
decent
superb
plenty
lively
outstanding
enjoy
awesome
fabulous
solid
incredible
phenomenal
brilliant
notch
beat
diminished
helpfull
foley


In [335]:
for i, word in enumerate(words):
    score = model.similarity(word,'disappointed')
    if score > 0.55:
        print(word)

/Users/allan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


disappointed
reading
based
excited
impressed
disappointing
rave
underwhelmed


# Negation

In [ ]:
#载入情感词典
posdict=[]
negdict=[]
verydict=[]
insufficientdict=[]
inverdict=[]
pos=open('/Users/RyanYang/PycharmProjects/linguistic_analysis/sentiment_dictionary/positive.txt')
for line in pos:
    posdict.append(line.rstrip('\n'))
neg=open('/Users/RyanYang/PycharmProjects/linguistic_analysis/sentiment_dictionary/negative.txt', 'rU')
for line in neg:
    negdict.append(line.rstrip('\n'))
very=open('/Users/RyanYang/PycharmProjects/linguistic_analysis/sentiment_dictionary/very.txt', 'rU')
for line in very:
    verydict.append(line.rstrip('\n'))
insufficient=open('/Users/RyanYang/PycharmProjects/linguistic_analysis/sentiment_dictionary/insufficient.txt', 'rU')
for line in insufficient:
    insufficientdict.append(line.rstrip('\n'))
inver=open('/Users/RyanYang/PycharmProjects/linguistic_analysis/sentiment_dictionary/inverse.txt', 'rU')
for line in inver:
    inverdict.append(line.rstrip('\n'))

In [ ]:
from __future__ import division
import csv
import time

start_time = time.clock()



#载入评论
review = open('/Users/RyanYang/PycharmProjects/linguistic_analysis/Valence/extract_data/value_2016.txt', 'rU')

#判定奇数偶数时使用:
def judgeodd(num):
    if num & 1 == 0:
        return 'even'
    else:
        return 'odd'

#主程序
def sentiment_score_list(dataset):
    cuted_data=[]
    for cell in dataset:
        cuted_data.append(cell.split())

    count1=[]
    count2=[]
    count3=[]

    for sent in cuted_data:
        len_sent = len(sent)
        i=0  # 扫描词的位置
        a=0
        poscount = 0  # 积极词的第一次分值
        poscount2 = 0  # 积极词反转后的分值
        poscount3 = 0  # 积极词的最后分值（包括叹号的分值）
        negcount = 0
        negcount2 = 0
        negcount3 = 0
        for word in sent:
            if word in posdict:
                poscount += 1
                c = 0
                for w in sent[a:i]:
                    if w in verydict:
                        poscount *= 2.0
                    elif w in insufficientdict:
                        poscount /= 2.0
                    elif w in inverdict:
                        c += 1
                if judgeodd(c) == 'odd':  # 如果情感词前inverse次数为奇数
                    poscount *= -1.0
                    poscount2 += poscount
                    poscount = 0
                    poscount3 += poscount2
                    poscount2 = 0

                else:
                    poscount3 += poscount
                    poscount = 0

                a = i + 1  # 情感词位置前移

            elif word in negdict:
                negcount += 1
                d=0
                for w in sent[a:i]:
                    if w in verydict:
                        negcount *= 2.0
                    elif w in insufficientdict:
                        negcount /= 2.0
                    elif w in inverdict:
                        d += 1

                if judgeodd(d) == 'odd':  # 如果情感词前inverse次数为奇数
                    negcount *= -1.0
                    negcount2 += negcount
                    negcount = 0
                    negcount3 += negcount2
                    negcount2 = 0

                else:
                    negcount3 += negcount
                    negcount = 0

                a = i + 1

            i += 1  # 扫描位置前移

        pos_count = 0
        neg_count = 0
        if poscount3 < 0 and negcount3 >= 0:
            neg_count = negcount3 - poscount3
            pos_count = 0
        elif negcount3 < 0 and poscount3 > 0:
            pos_count = poscount3 - negcount3
            neg_count = 0
        elif poscount3 < 0 and negcount3 < 0:
            neg_count = -poscount3
            pos_count = -negcount3
        else:
            pos_count = poscount3
            neg_count = negcount3

        score = pos_count-neg_count
        final_score = score/(len_sent)
        count1.append(final_score)
        count3.append(len_sent)

    count2.append(count1)
    #count1 = []

    return count1, count3

#def sentiment_score(senti_score_list):
#    score = []
#    for rev in senti_score_list:
#        score_array = np.array(rev)
#        Pos = np.sum(score_array[:, 0])
#        Neg = np.sum(score_array[:, 1])
#        AvgPos = np.mean(score_array[:, 0])
#        AvgNeg = np.mean(score_array[:, 1])
#        StdPos = np.std(score_array[:, 0])
#        StdNeg = np.std(score_array[:, 1])
#        score.append([Pos, Neg, AvgPos, AvgNeg, StdPos, StdNeg])
#    return score

sentiment_list = sentiment_score_list(review)
sentiment = sentiment_list[0]
length = sentiment_list[1]


with open ('value_valence_2016.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    for row in zip(sentiment, length):
        writer.writerow(row)


#final_score = sentiment_score (sentiment_list)
print(time.clock() - start_time, "seconds")
#print final_score

# count skills 

In [95]:
from collections import Counter
a = Counter(service_word_list)

In [103]:
service_result = pd.value_counts(service_word_list)
service_result.head(50)

service          24
slammed          21
personable       21
incredibly       21
chummy           21
extremely        21
polite           21
russia           20
waitstaff        20
smiling          20
manner           20
timely           20
sever            19
buzzy            19
hebert           19
waitresses       19
servers          19
informative      19
patient          19
staffs           19
timing           19
cheerful         19
organized        19
waiters          19
unobtrusive      18
sevice           18
humored          18
engaging         18
busboys          18
professional     18
jalisco          18
greeting         18
jobs             18
intrusive        18
hovering         18
competent        18
goran            18
schedule         18
paced            18
overbearing      18
funny            18
teamwork         18
chatty           18
knowledgable     18
advising         18
shannon          18
courteous        18
hostesses        17
answering        17
accommodating    17
